In [1]:
import os
import kagglehub
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
import cv2
from src import ImageClassifier

/Users/vladyoslav/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset
path = os.path.join(
    kagglehub.dataset_download("rahmasleam/intel-image-dataset"), "Intel Image Dataset"
)
print("Dataset path:", path)

Dataset path: /Users/vladyoslav/.cache/kagglehub/datasets/rahmasleam/intel-image-dataset/versions/1/Intel Image Dataset


In [3]:
# Get list of categories
categories = list(filter(lambda category: category != ".DS_Store", os.listdir(path)))
print("Categories:", categories)

Categories: ['forest', 'buildings', 'glacier', 'street', 'mountain', 'sea']


In [4]:
# Function to get image arrays and labels
def get_image_arrays(dataset_path):
    """
    Retrieve the image arrays and their corresponding labels.

    Args:
        dataset_path (str): Path to the root folder containing images.

    Returns:
        tuple: A tuple containing two lists:
            - List of image arrays (np.ndarray).
            - List of labels for each image.
    """
    images = []
    labels = []

    for category in categories:
        category_path = os.path.join(dataset_path, category)
        for img_file in os.listdir(category_path):
            img_path = os.path.join(category_path, img_file)
            if os.path.isfile(img_path) and img_path.lower().endswith(
                (".png", ".jpg", ".jpeg")
            ):
                image = cv2.imread(img_path)
                if image is not None:  # Ensure the image was successfully loaded
                    images.append(image)
                    labels.append(category)

    return images, labels

In [5]:
from sklearn.preprocessing import StandardScaler
# Get all image arrays and labels
images, labels = get_image_arrays(path)

# Split the dataset into training and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=52
)

# Initialize the classifier
classifier = ImageClassifier()

# Train the model on the training set
classifier.fit(train_images, train_labels)

Extracting features from images...
Training the Random Forest classifier...
Training complete!


In [9]:
# Predict and evaluate the F1 score on the test set
y_true = []
y_pred = []
category_stats = {
    category: {"total": 0, "correct": 0, "incorrect": 0} for category in categories
}

# Loop through the test images and predict their categories
for img, true_label in tqdm(zip(test_images, test_labels), total=len(test_images)):
    # Get predicted probabilities and categorize the result
    predicted_probs = classifier.predict(img)
    predicted_label = max(predicted_probs, key=predicted_probs.get)

    # Add true and predicted labels to lists
    y_true.append(true_label)
    y_pred.append(predicted_label)

    # Update category statistics
    category_stats[true_label]["total"] += 1
    if predicted_label == true_label:
        category_stats[true_label]["correct"] += 1
    else:
        category_stats[true_label]["incorrect"] += 1

  0%|          | 0/600 [00:00<?, ?it/s]

Extracting features for prediction...
{'forest': np.float64(0.26), 'buildings': np.float64(0.06), 'glacier': np.float64(0.11), 'street': np.float64(0.03), 'mountain': np.float64(0.07), 'sea': np.float64(0.47)}
sea


In [7]:
# Calculate F1 score
f1 = f1_score(y_true, y_pred, average="weighted")
print(f"Overall F1 score: {f1:.4f}")

Overall F1 score: 0.1691


In [8]:
# Prepare the category-wise report
report_data = []
for category in categories:
    total = category_stats[category]["total"]
    correct = category_stats[category]["correct"]
    incorrect = category_stats[category]["incorrect"]
    accuracy_category = correct / total if total > 0 else 0
    report_data.append(
        {
            "Category": category,
            "Total": total,
            "Correct": correct,
            "Incorrect": incorrect,
            "(%)": accuracy_category,
        }
    )

# Convert to DataFrame for easy display
report_df = pd.DataFrame(report_data)

# Add a row for the total F1 score (this will not be a weighted score, so it's not directly comparable)
report_df.loc["Total"] = report_df.sum(numeric_only=True)
report_df.loc["Total", "Category"] = "Total"
report_df.loc["Total", "(%)"] = f1

# Print the report table
print(report_df)

        Category  Total  Correct  Incorrect       (%)
0         forest   97.0      2.0       95.0  0.020619
1      buildings   89.0      6.0       83.0  0.067416
2        glacier  102.0     75.0       27.0  0.735294
3         street  110.0      4.0      106.0  0.036364
4       mountain  110.0     12.0       98.0  0.109091
5            sea   92.0      6.0       86.0  0.065217
Total      Total  600.0    105.0      495.0  0.169079
